<a href="https://colab.research.google.com/github/hruthiksiva/DataScience-Competitions/blob/main/enigma_datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extract csv

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1XcMvwBxCa8WDRBbA-UsTK4T8gAyA42SP
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c domain-network-security

Downloading...
From: https://drive.google.com/u/0/uc?id=1XcMvwBxCa8WDRBbA-UsTK4T8gAyA42SP
To: /content/kaggle.json
100% 70.0/70.0 [00:00<00:00, 98.0kB/s]
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample%20submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip *.zip  && rm *.zip

Archive:  train.csv.zip
  inflating: train.csv               


## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Collection/ Alteration

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45870 entries, 0 to 45869
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    45870 non-null  int64  
 1   Source Port           36683 non-null  float64
 2   Destination Port      36693 non-null  float64
 3   NAT Source Port       36756 non-null  float64
 4   NAT Destination Port  36674 non-null  float64
 5   Bytes                 36607 non-null  float64
 6   Bytes Sent            36764 non-null  float64
 7   Bytes Received        36693 non-null  float64
 8   Packets               36818 non-null  float64
 9   Elapsed Time (sec)    36584 non-null  float64
 10  pkts_sent             36667 non-null  float64
 11  pkts_received         36711 non-null  float64
 12  Action                45870 non-null  object 
dtypes: float64(11), int64(1), object(1)
memory usage: 4.5+ MB


In [ ]:
df.head()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action,port_same
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,1.0,1.0,allow,1
7,64974,50472.0,445.0,0.0,0.0,66.0,66.0,0.0,1.0,0.0,1.0,0.0,drop,0
8,41097,56832.0,445.0,0.0,0.0,70.0,70.0,0.0,1.0,0.0,1.0,0.0,drop,0
9,10866,64723.0,445.0,0.0,0.0,70.0,70.0,0.0,1.0,0.0,1.0,0.0,drop,0
11,46585,56076.0,443.0,18045.0,443.0,1786.0,172.0,1614.0,7.0,120.0,3.0,4.0,allow,1


In [ ]:
df['port_same'] = (df['Destination Port'] == df['NAT Destination Port']).astype(int)

In [ ]:
df['port_same'].value_counts()

1    6776
0    5249
Name: port_same, dtype: int64

In [ ]:
a={445:0,443:443,80:80,123:123,5484:5484,0:0,53:53}

In [ ]:
for i,j in zip(a.keys(),a.values()):
  print(i,j)

445 0
443 443
80 80
123 123
5484 5484
0 0
53 53


In [ ]:
df=df.dropna()
df[(df['NAT Source Port'] == 0) & (df['NAT Destination Port'] != 0)].shape

(19, 13)

In [ ]:
df=df.dropna()
df[(df['Bytes Sent']==0)].shape

(0, 13)

In [ ]:

df[(df['Source Port']==df['NAT Source Port']) & (df['Destination Port']!=df['NAT Destination Port'])].shape

(0, 13)

In [ ]:
df['Action'].unique()

array(['allow', 'drop', 'deny', 'reset-both'], dtype=object)

In [ ]:
df[df['Destination Port'] < 80]['Destination Port'].unique()

array([53.,  0., 23.,  1., 21., 37., 67., 66., 25., 17., 22., 68.,  7.])

In [ ]:
sum(df.dropna()['Packets'] - df.dropna()['pkts_sent'] - df.dropna()['pkts_received'] != 0)

0

In [ ]:
def cusFil(x):
  a,b,c = x[0],x[1],x[2]
  if np.isnan(a) and (not np.isnan(b)) and (not np.isnan(c)):
      a=b+c
  elif (not np.isnan(a)) and np.isnan(b) and (not np.isnan(c)):
      b=a-c
  elif (not np.isnan(a)) and (not np.isnan(b)) and np.isnan(c):
      c=a-b
  x[0],x[1],x[2] = a,b,c
  return x

In [ ]:
ll={445:0,443:443,80:80,123:123,5484:5484,53:53}
ll2={443:443,80:80,123:123,5484:5484,53:53}
def cusFil2(x):
    a,b,c,d = x[0],x[1],x[2],x[3]
    if np.isnan(b) or np.isnan(d):
        if np.isnan(b):
            if d in ll2:
                b=ll2[d]
        else:
            if b in ll:
                d=ll[b]
    x[0],x[1],x[2],x[3] = a,b,c,d
    return x

def cusFil3(x):
    a,b,c,d = x[0],x[1],x[2],x[3]
    if a == c:
        if np.isnan(b) or np.isnan(d):
            if np.isnan(b):
                b=d
            else:
                d=b
    if b == d:
        if np.isnan(a) or np.isnan(c):
            if np.isnan(a):
                a=c
            else:
                c=a
    x[0],x[1],x[2],x[3] = a,b,c,d
    return x

def cusFil4(x):
    a,b = x[0],x[1]
    if a == 0 or b == 0:
        a,b=0.,0.0
    x[0],x[1] = a,b
    return x

In [ ]:
0 == 0.0

True

In [ ]:
df = pd.read_csv('test.csv')

In [ ]:
df[['Source Port', 'Destination Port', 'NAT Source Port','NAT Destination Port']] = df[['Source Port', 'Destination Port', 'NAT Source Port','NAT Destination Port']].apply(cusFil2,axis=1)
df[['Source Port', 'Destination Port', 'NAT Source Port','NAT Destination Port']] = df[['Source Port', 'Destination Port', 'NAT Source Port','NAT Destination Port']].apply(cusFil3,axis=1)

In [ ]:
df[['Bytes','Bytes Sent','Bytes Received']]=df[['Bytes','Bytes Sent','Bytes Received']].apply(cusFil,axis=1)
df[['Packets','pkts_sent','pkts_received']]=df[['Packets','pkts_sent','pkts_received']].apply(cusFil,axis=1)

In [ ]:
df[['Bytes Received','pkts_received']] = df[['Bytes Received','pkts_received']].apply(cusFil4,axis=1)
df[['Bytes Sent','pkts_sent']] = df[['Bytes Sent','pkts_sent']].apply(cusFil4,axis=1)

In [ ]:
df[df['Bytes Received'].isnull()]

In [ ]:
df.to_csv('test1.csv', index=False)

In [ ]:
pd.read_csv('test.csv').dropna()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
9,32524,42317.0,443.0,11761.0,443.0,416.0,210.0,206.0,7.0,27.0,4.0,3.0
11,30700,55489.0,53.0,44022.0,53.0,185.0,102.0,83.0,2.0,30.0,1.0,1.0
40,13846,51221.0,57073.0,10928.0,57073.0,481.0,150.0,331.0,2.0,1200.0,1.0,1.0
42,58159,64823.0,53.0,51719.0,53.0,199.0,102.0,97.0,2.0,30.0,1.0,1.0
44,62234,51233.0,445.0,0.0,0.0,66.0,66.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19619,12718,49982.0,53.0,16411.0,53.0,211.0,110.0,101.0,2.0,30.0,1.0,1.0
19620,52028,54038.0,445.0,0.0,0.0,66.0,66.0,0.0,1.0,0.0,1.0,0.0
19634,11790,55290.0,53.0,19726.0,53.0,198.0,94.0,104.0,2.0,30.0,1.0,1.0
19644,37225,30960.0,45732.0,62764.0,45732.0,330.0,330.0,0.0,6.0,3610.0,6.0,0.0


In [ ]:
for i in df.columns:
  print(df[df[i].isnull()].shape)

(0, 12)
(2317, 12)
(2455, 12)
(2395, 12)
(1782, 12)
(1148, 12)
(1129, 12)
(763, 12)
(1136, 12)
(3959, 12)
(1110, 12)
(813, 12)


In [ ]:
df.isnull().sum()

ID                         0
Source Port             9187
Destination Port        9177
NAT Source Port         9114
NAT Destination Port    9196
Bytes                   9263
Bytes Sent              9106
Bytes Received          9177
Packets                 9052
Elapsed Time (sec)      9286
pkts_sent               9203
pkts_received           9159
Action                     0
dtype: int64

In [ ]:
df=pd.read_csv('train1.csv')
df2=pd.read_csv('test1.csv')

In [ ]:
df.head()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,0
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,1.0,0.0,1.0,0.0,1
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,1.0,1.0,0
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,2
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,1.0,1.0,0


### Encoding Data

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
df['Action']= lb.fit_transform(df['Action'])

In [ ]:
X,y=df.drop(columns=['Action']),df[['Action']]

In [ ]:
df['Action'].describe()

count    45870.000000
mean         0.623305
std          0.794887
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          3.000000
Name: Action, dtype: float64

### Nan Count and Removal <3

In [ ]:
for i in range(len(df)) :
        df.loc[i,"nancount"]= df.iloc[i].isnull().sum()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action,nancount
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,0,1.0
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,1,3.0
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,0,2.0
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,2,3.0
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45865,36322,NaN,53.0,NaN,53.0,94.0,94.0,0.0,NaN,29.0,NaN,NaN,0,6.0
45866,36418,NaN,53.0,44635.0,53.0,177.0,94.0,83.0,2.0,NaN,NaN,1.0,0,4.0
45867,2289,50438.0,445.0,0.0,0.0,70.0,NaN,0.0,1.0,0.0,1.0,0.0,2,2.0
45868,10229,37785.0,80.0,22917.0,80.0,2293.0,1576.0,NaN,12.0,15.0,6.0,NaN,0,3.0


### Float to Int

In [ ]:
list =['Source Port','Destination Port','NAT Source Port','NAT Destination Port','Bytes',
       'Bytes Sent','Bytes Received','Packets','Elapsed Time (sec)','pkts_sent','pkts_received','nancount']

for i in list:
  df[str(i)] = df[str(i)].fillna(-1)
  df[str(i)]=df[str(i)].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45870 entries, 0 to 45869
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   ID                    45870 non-null  int64
 1   Source Port           45870 non-null  int64
 2   Destination Port      45870 non-null  int64
 3   NAT Source Port       45870 non-null  int64
 4   NAT Destination Port  45870 non-null  int64
 5   Bytes                 45870 non-null  int64
 6   Bytes Sent            45870 non-null  int64
 7   Bytes Received        45870 non-null  int64
 8   Packets               45870 non-null  int64
 9   Elapsed Time (sec)    45870 non-null  int64
 10  pkts_sent             45870 non-null  int64
 11  pkts_received         45870 non-null  int64
 12  Action                45870 non-null  int64
 13  nancount              45870 non-null  int64
dtypes: int64(14)
memory usage: 4.9 MB


### Separate Label vs Feature

In [ ]:
y = df.loc[:,'Action']
X = df.drop(['ID','Action'],axis=1)

## Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Data Visualization

In [ ]:
print(df['ID'].corr(df['Source Port']))
print(df['pkts_received'].corr(df['Action']))
print(df['ID'].corr(df['Action']))

0.004004893499190068
-0.018264771302748367
-6.193059199356068e-05


In [ ]:
df[df['Action']==3].describe()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action,nancount
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.0,37.000000
mean,29963.783784,21487.378378,13780.216216,2297.837838,274.405405,122.756757,111.918919,11.216216,0.864865,-0.081081,0.621622,-0.081081,3.0,3.513514
std,15989.087211,19801.821543,16836.519652,7781.713584,937.934180,94.790531,70.761877,26.220364,1.530471,0.546625,1.036858,0.546625,0.0,1.366700
min,229.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3.0,1.000000
25%,20320.000000,6889.000000,-1.000000,-1.000000,-1.000000,62.000000,62.000000,0.000000,-1.000000,0.000000,1.000000,0.000000,3.0,3.000000
50%,29613.000000,14967.000000,3389.000000,0.000000,0.000000,143.000000,143.000000,0.000000,1.000000,0.000000,1.000000,0.000000,3.0,3.000000
75%,39210.000000,29951.000000,25773.000000,0.000000,0.000000,157.000000,146.000000,0.000000,1.000000,0.000000,1.000000,0.000000,3.0,4.000000
max,57763.000000,64461.000000,61248.000000,38126.000000,3389.000000,293.000000,223.000000,70.000000,4.000000,1.000000,3.000000,1.000000,3.0,8.000000


# Model Decision

### LightGBM

In [ ]:
import lightgbm as lgb

### KNN Algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

### XGBooster

In [ ]:
import xgboost as xgb
xgb.XGBClassifier(missing=-1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=-1, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
model=xgb.XGBClassifier(missing=np.nan,n_estimators=100,max_depth=20,learning_rate=0.5)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier()
model .fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Prediction

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test, y_pred))
#xgbooster - 0.985
#random forest - 0.984
#knn - 0.95

Accuracy: 0.988009592326139


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')

0.9918875038986195

# Data Extraction

In [ ]:
tdf=pd.read_csv('test.csv')

In [ ]:
le=LabelEncoder()
df['Action']= le.fit_transform(df['Action'])

In [ ]:
list =['Source Port','Destination Port','NAT Source Port','NAT Destination Port','Bytes',
       'Bytes Sent','Bytes Received','Packets','Elapsed Time (sec)','pkts_sent','pkts_received']

for i in list:
  tdf[str(i)] = tdf[str(i)].fillna(-1)
  tdf[str(i)]=tdf[str(i)].astype(int)
tdf.info()

In [ ]:
X_=tdf.drop(y.columns,axis = 1)

In [ ]:
ypred_=model.predict(X_)

In [ ]:
ypred_ = le.inverse_transform(ypred_)

In [ ]:
#dont forget to decode labels back to open closed etc
sub=pd.DataFrame({"id":tdf["id"],"target":np.array(ypred_).reshape(-1)})
sub.to_csv("submission1.csv",index=False)